# Prepare Chicago Crime Data for a GitHub Repository

- Original Notebook Source: https://github.com/coding-dojo-data-science/preparing-chicago-crime-data
- Updated 11/17/22

>- This notebook will process a "Crimes - 2001 to Preset.csv" crime file in your Downloads folder and save it as smaller .csv's in a new "Data/Chicago/" folder inside this notebook's folder/repo.

# INSTRUCTIONS

- 1) Go to the Chicago Data Portal's page for ["Crimes - 2001 to Preset"](https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-Present/ijzp-q8t2).

- 2) Click on the Export button on the top right and select CSV. 
    - Save the file to your Downloads folder instead of your repository. **The file is too big for a repository.**
    
    
    
- 3) Wait for the full file to download. 
    - It is very large (over >1.7GB and may take several minutes to fully download.)
    
    
- 4) Once the download is complete, change `RAW_FILE` variable below to match the filepath to the downloaded file.

## 🚨 Set the correct `RAW_FILE` path

- The cell below will attempt to check your Downloads folder for any file with a name that contains "Crimes_-_2001_to_Present".
    - If you know the file path already, you can skip the next cell and just manually set the RAW_FILE variable in the following code cell.

In [21]:
## Run the cell below to attempt to programmatically find your crime file
import os,glob

## Getting the home folder from environment variables
home_folder = os.environ['HOME']
# print("- Your Home Folder is: " + home_folder)

## Check for downloads folder
if 'Documents' in os.listdir(home_folder):
    
    
    # Print the Downloads folder path
    dl_folder = os.path.abspath(os.path.join(home_folder,'Documents'))
    print(f"- Your Documents folder is '{dl_folder}/'\n")
    
    ## checking for crime files using glob
    crime_files = sorted(glob.glob(dl_folder+'/**/Crimes_-_2001_to_Present*',recursive=True))
    
    # If more than 
    if len(crime_files)==1:
        RAW_FILE = crime_files[0]
        
    elif len(crime_files)>1:
        print('[i] The following files were found:')
        
        for i, fname in enumerate(crime_files):
            print(f"\tcrime_files[{i}] = '{fname}'")
        print(f'\n- Please fill in the RAW_FILE variable in the code cell below with the correct filepath.')

else:
    print(f'[!] Could not programmatically find your downloads folder.')
    print('- Try using Finder (on Mac) or File Explorer (Windows) to navigate to your Downloads folder.')


- Your Documents folder is 'C:\Users\caill\Documents/'



In [22]:
## (Required) MAKE SURE TO CHANGE THIS VARIABLE TO MATCH YOUR LOCAL FILE NAME
#(or slice correct index from the crime_files list)

RAW_FILE = r"C:\Windows\System32\Chicago-Crime-Data\Data\Chicago\Crimes_-_2001_to_Present.csv"

if RAW_FILE ==  "YOUR FILEPATH HERE" :
    raise Exception("You must update the RAW_FILE variable to match your local filepath.")

RAW_FILE

'C:\\Windows\\System32\\Chicago-Crime-Data\\Data\\Chicago\\Crimes_-_2001_to_Present.csv'

In [3]:
## (Optional) SET THE FOLDER FOR FINAL FILES
OUTPUT_FOLDER = 'Data/Chicago/'
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# 🔄 Full Workflow

- Now that your RAW_FILE variable is set either:
    - On the toolbar, click on the Kernel menu > "Restart and Run All".
    - OR click on this cell first, then on the toolbar click on the "Cell" menu > "Run All Below"

In [23]:
import pandas as pd
pd.set_option('display.max_columns', 100)
pd.set_option('display.float_format',lambda x: f"{x:,.2f}")

In [25]:
from tqdm import tqdm
import time

# Define the total number of iterations
total_iterations = 100

# Create a loop with tqdm to track progress
for i in tqdm(range(total_iterations)):
    # Simulate some work being done
    time.sleep(0.1)
    
    chicago_full = pd.read_csv(RAW_FILE)

100%|████████████████████████████████████████████████████████████████████████████| 100/100 [19:19:10<00:00, 695.51s/it]


In [26]:
# explicitly setting the format to speed up pd.to_datetime
date_format = "%m/%d/%Y %H:%M:%S %p"


### Demonstrating/testing date_format
example = chicago_full.loc[0,'Date']
display(example)
pd.to_datetime(example,format=date_format)

'09/01/2018 12:01:00 AM'

Timestamp('2018-09-01 12:01:00')

In [29]:
chicago_full.reset_index(drop = True, inplace = True)

In [30]:
%%time
# this cell can take up to 1 min to run
chicago_full['Datetime'] = pd.to_datetime(chicago_full['Date'], format=date_format)
chicago_full = chicago_full.sort_values('Datetime')
chicago_full = chicago_full.set_index('Datetime')
chicago_full

CPU times: total: 36.8 s
Wall time: 5min 22s


,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
Datetime,,,,,,,,,,,,,,,,,,,,,,
2001-01-01 01:00:00,1358218,G059453,01/01/2001 01:00:00 PM,063XX S RHODES AV,0560,ASSAULT,SIMPLE,APARTMENT,False,False,312,3.00,NaN,NaN,08A,"1,180,959.00","1,862,790.00",2001,08/17/2015 03:03:40 PM,41.78,-87.61,"(41.778744378, -87.612135367)"
2001-01-01 01:00:00,1310288,G000636,01/01/2001 01:00:00 AM,075XX S UNION AV,1310,CRIMINAL DAMAGE,TO PROPERTY,RESIDENCE,False,False,621,6.00,NaN,NaN,14,"1,172,985.00","1,854,673.00",2001,08/17/2015 03:03:40 PM,41.76,-87.64,"(41.756650158, -87.641607815)"
2001-01-01 01:00:00,1310393,G000713,01/01/2001 01:00:00 AM,052XX N EAST RIVER RD,1320,CRIMINAL DAMAGE,TO VEHICLE,STREET,False,False,1614,16.00,NaN,NaN,14,"1,116,671.00","1,933,824.00",2001,08/17/2015 03:03:40 PM,41.97,-87.85,"(41.974911416, -87.846347904)"
2001-01-01 01:00:00,1316324,G002222,01/01/2001 01:00:00 PM,059XX W ARTHINGTON ST,0810,THEFT,OVER $500,STREET,False,False,1513,15.00,NaN,NaN,06,"1,136,610.00","1,895,353.00",2001,08/17/2015 03:03:40 PM,41.87,-87.77,"(41.869008134, -87.77394715)"
2001-01-01 01:00:00,1311626,G001009,01/01/2001 01:00:00 AM,023XX S TROY ST,1320,CRIMINAL DAMAGE,TO VEHICLE,STREET,False,False,1033,10.00,NaN,NaN,14,"1,155,692.00","1,888,116.00",2001,08/17/2015 03:03:40 PM,41.85,-87.70,"(41.848786421, -87.704086603)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-30 12:50:00,13124429,JG321799,06/30/2023 12:50:00 AM,036XX N LAKE SHORE DR,0910,MOTOR VEHICLE THEFT,AUTOMOBILE,STREET,False,False,1925,19.00,46.00,6.00,07,"1,171,841.00","1,924,794.00",2023,07/07/2023 04:48:05 PM,41.95,-87.64,"(41.949092979, -87.643736581)"
2023-06-30 12:50:00,13124821,JG322246,06/30/2023 12:50:00 PM,010XX W BELMONT AVE,0860,THEFT,RETAIL THEFT,TAVERN / LIQUOR STORE,False,False,1924,19.00,44.00,6.00,06,"1,168,964.00","1,921,423.00",2023,07/07/2023 04:48:05 PM,41.94,-87.65,"(41.939905846, -87.654410045)"
2023-06-30 12:52:00,13124281,JG321614,06/30/2023 12:52:00 AM,043XX W NORTH AVE,1570,SEX OFFENSE,PUBLIC INDECENCY,STREET,False,False,2534,25.00,36.00,23.00,17,"1,146,899.00","1,910,274.00",2023,07/07/2023 04:48:05 PM,41.91,-87.74,"(41.90976283, -87.73579178)"


In [31]:
(chicago_full.isna().sum()/len(chicago_full)).round(2)

ID                     0.00
Case Number            0.00
Date                   0.00
Block                  0.00
IUCR                   0.00
Primary Type           0.00
Description            0.00
Location Description   0.00
Arrest                 0.00
Domestic               0.00
Beat                   0.00
District               0.00
Ward                   0.08
Community Area         0.08
FBI Code               0.00
X Coordinate           0.01
Y Coordinate           0.01
Year                   0.00
Updated On             0.00
Latitude               0.01
Longitude              0.01
Location               0.01
dtype: float64

## Separate the Full Dataset by Years

In [32]:
# save the years for every crime
chicago_full["Year"] = chicago_full.index.year
chicago_full["Year"] = chicago_full["Year"].astype(str)
chicago_full["Year"].value_counts()

2002    486804
2001    485885
2003    475983
2004    469421
2005    453772
2006    448177
2007    437087
2008    427169
2009    392824
2010    370502
2011    351973
2012    336272
2013    307475
2014    275742
2016    269805
2017    269092
2018    268804
2015    264773
2019    261286
2022    238663
2020    212160
2021    208700
2023    123550
Name: Year, dtype: int64

In [33]:
## Dropping unneeded columns to reduce file size
drop_cols = ["X Coordinate","Y Coordinate", "Community Area","FBI Code",
             "Case Number","Updated On",'Block','Location','IUCR']

In [34]:
# save final df
chicago_final = chicago_full.drop(columns=drop_cols).sort_index()#.reset_index()
chicago_final

,ID,Date,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Year,Latitude,Longitude
Datetime,,,,,,,,,,,,,
2001-01-01 01:00:00,1358218,01/01/2001 01:00:00 PM,ASSAULT,SIMPLE,APARTMENT,False,False,312,3.00,NaN,2001,41.78,-87.61
2001-01-01 01:00:00,1310288,01/01/2001 01:00:00 AM,CRIMINAL DAMAGE,TO PROPERTY,RESIDENCE,False,False,621,6.00,NaN,2001,41.76,-87.64
2001-01-01 01:00:00,1310393,01/01/2001 01:00:00 AM,CRIMINAL DAMAGE,TO VEHICLE,STREET,False,False,1614,16.00,NaN,2001,41.97,-87.85
2001-01-01 01:00:00,1316324,01/01/2001 01:00:00 PM,THEFT,OVER $500,STREET,False,False,1513,15.00,NaN,2001,41.87,-87.77
2001-01-01 01:00:00,1311626,01/01/2001 01:00:00 AM,CRIMINAL DAMAGE,TO VEHICLE,STREET,False,False,1033,10.00,NaN,2001,41.85,-87.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-30 12:50:00,13124429,06/30/2023 12:50:00 AM,MOTOR VEHICLE THEFT,AUTOMOBILE,STREET,False,False,1925,19.00,46.00,2023,41.95,-87.64
2023-06-30 12:50:00,13124821,06/30/2023 12:50:00 PM,THEFT,RETAIL THEFT,TAVERN / LIQUOR STORE,False,False,1924,19.00,44.00,2023,41.94,-87.65
2023-06-30 12:52:00,13124281,06/30/2023 12:52:00 AM,SEX OFFENSE,PUBLIC INDECENCY,STREET,False,False,2534,25.00,36.00,2023,41.91,-87.74


In [35]:
chicago_final.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 7835919 entries, 2001-01-01 01:00:00 to 2023-06-30 12:58:00
Data columns (total 13 columns):
 #   Column                Dtype  
---  ------                -----  
 0   ID                    int64  
 1   Date                  object 
 2   Primary Type          object 
 3   Description           object 
 4   Location Description  object 
 5   Arrest                bool   
 6   Domestic              bool   
 7   Beat                  int64  
 8   District              float64
 9   Ward                  float64
 10  Year                  object 
 11  Latitude              float64
 12  Longitude             float64
dtypes: bool(2), float64(4), int64(2), object(5)
memory usage: 732.3+ MB


In [36]:
chicago_final.memory_usage(deep=True).astype(float)

Index                   62,687,352.00
ID                      62,687,352.00
Date                   619,037,601.00
Primary Type           525,336,482.00
Description            573,572,596.00
Location Description   534,118,936.00
Arrest                   7,835,919.00
Domestic                 7,835,919.00
Beat                    62,687,352.00
District                62,687,352.00
Ward                    62,687,352.00
Year                   477,991,059.00
Latitude                62,687,352.00
Longitude               62,687,352.00
dtype: float64

In [37]:
# unique # of year bins
year_bins = chicago_final['Year'].astype(str).unique()
year_bins

array(['2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016',
       '2017', '2018', '2019', '2020', '2021', '2022', '2023'],
      dtype=object)

In [38]:
FINAL_DROP = ['Datetime','Year']#,'Location Description']

In [39]:
## set save location 

os.makedirs(OUTPUT_FOLDER, exist_ok=True)
print(f"[i] Saving .csv's to {OUTPUT_FOLDER}")
## loop through years
for year in year_bins:
    
    ## save temp slices of dfs to save.
    temp_df = chicago_final.loc[ year]
    temp_df = temp_df.reset_index(drop=False)
    temp_df = temp_df.drop(columns=FINAL_DROP)

    # save as csv to output folder
    fname_temp = f"{OUTPUT_FOLDER}Chicago-Crime_{year}.csv"#.gz
    temp_df.to_csv(fname_temp,index=False)

    print(f"- Succesfully saved {fname_temp}")

[i] Saving .csv's to Data/Chicago/
- Succesfully saved Data/Chicago/Chicago-Crime_2001.csv
- Succesfully saved Data/Chicago/Chicago-Crime_2002.csv
- Succesfully saved Data/Chicago/Chicago-Crime_2003.csv
- Succesfully saved Data/Chicago/Chicago-Crime_2004.csv
- Succesfully saved Data/Chicago/Chicago-Crime_2005.csv
- Succesfully saved Data/Chicago/Chicago-Crime_2006.csv
- Succesfully saved Data/Chicago/Chicago-Crime_2007.csv
- Succesfully saved Data/Chicago/Chicago-Crime_2008.csv
- Succesfully saved Data/Chicago/Chicago-Crime_2009.csv
- Succesfully saved Data/Chicago/Chicago-Crime_2010.csv
- Succesfully saved Data/Chicago/Chicago-Crime_2011.csv
- Succesfully saved Data/Chicago/Chicago-Crime_2012.csv
- Succesfully saved Data/Chicago/Chicago-Crime_2013.csv
- Succesfully saved Data/Chicago/Chicago-Crime_2014.csv
- Succesfully saved Data/Chicago/Chicago-Crime_2015.csv
- Succesfully saved Data/Chicago/Chicago-Crime_2016.csv
- Succesfully saved Data/Chicago/Chicago-Crime_2017.csv
- Succesfully

In [40]:
saved_files = sorted(glob.glob(OUTPUT_FOLDER+'*.*csv'))
saved_files

['Data/Chicago\\Chicago-Crime_2001.csv',
 'Data/Chicago\\Chicago-Crime_2002.csv',
 'Data/Chicago\\Chicago-Crime_2003.csv',
 'Data/Chicago\\Chicago-Crime_2004.csv',
 'Data/Chicago\\Chicago-Crime_2005.csv',
 'Data/Chicago\\Chicago-Crime_2006.csv',
 'Data/Chicago\\Chicago-Crime_2007.csv',
 'Data/Chicago\\Chicago-Crime_2008.csv',
 'Data/Chicago\\Chicago-Crime_2009.csv',
 'Data/Chicago\\Chicago-Crime_2010.csv',
 'Data/Chicago\\Chicago-Crime_2011.csv',
 'Data/Chicago\\Chicago-Crime_2012.csv',
 'Data/Chicago\\Chicago-Crime_2013.csv',
 'Data/Chicago\\Chicago-Crime_2014.csv',
 'Data/Chicago\\Chicago-Crime_2015.csv',
 'Data/Chicago\\Chicago-Crime_2016.csv',
 'Data/Chicago\\Chicago-Crime_2017.csv',
 'Data/Chicago\\Chicago-Crime_2018.csv',
 'Data/Chicago\\Chicago-Crime_2019.csv',
 'Data/Chicago\\Chicago-Crime_2020.csv',
 'Data/Chicago\\Chicago-Crime_2021.csv',
 'Data/Chicago\\Chicago-Crime_2022.csv',
 'Data/Chicago\\Chicago-Crime_2023.csv']

In [41]:
## create a README.txt for the zip files
readme = """Source URL: 
- https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-Present/ijzp-q8t2
- Filtered for years 2000-Present.

Downloaded 07/18/2022
- Files are split into 1 year per file.

EXAMPLE USAGE:
>> import glob
>> import pandas as pd
>> folder = "Data/Chicago/"
>> crime_files = sorted(glob.glob(folder+"*.csv"))
>> df = pd.concat([pd.read_csv(f) for f in crime_files])
"""
print(readme)


with open(f"{OUTPUT_FOLDER}README.txt",'w') as f:
    f.write(readme)

Source URL: 
- https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-Present/ijzp-q8t2
- Filtered for years 2000-Present.

Downloaded 07/18/2022
- Files are split into 1 year per file.

EXAMPLE USAGE:
>> import glob
>> import pandas as pd
>> folder = "Data/Chicago/"
>> crime_files = sorted(glob.glob(folder+"*.csv"))
>> df = pd.concat([pd.read_csv(f) for f in crime_files])



## Confirmation

- Follow the example usage above to test if your files were created successfully.

In [42]:
# get list of files from folder
crime_files = sorted(glob.glob(OUTPUT_FOLDER+"*.csv"))
df = pd.concat([pd.read_csv(f) for f in crime_files])
df

,ID,Date,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Latitude,Longitude
0,1358218,01/01/2001 01:00:00 PM,ASSAULT,SIMPLE,APARTMENT,False,False,312,3.00,NaN,41.78,-87.61
1,1310288,01/01/2001 01:00:00 AM,CRIMINAL DAMAGE,TO PROPERTY,RESIDENCE,False,False,621,6.00,NaN,41.76,-87.64
2,1310393,01/01/2001 01:00:00 AM,CRIMINAL DAMAGE,TO VEHICLE,STREET,False,False,1614,16.00,NaN,41.97,-87.85
3,1316324,01/01/2001 01:00:00 PM,THEFT,OVER $500,STREET,False,False,1513,15.00,NaN,41.87,-87.77
4,1311626,01/01/2001 01:00:00 AM,CRIMINAL DAMAGE,TO VEHICLE,STREET,False,False,1033,10.00,NaN,41.85,-87.70
...,...,...,...,...,...,...,...,...,...,...,...,...
123545,13124429,06/30/2023 12:50:00 AM,MOTOR VEHICLE THEFT,AUTOMOBILE,STREET,False,False,1925,19.00,46.00,41.95,-87.64
123546,13124821,06/30/2023 12:50:00 PM,THEFT,RETAIL THEFT,TAVERN / LIQUOR STORE,False,False,1924,19.00,44.00,41.94,-87.65
123547,13124281,06/30/2023 12:52:00 AM,SEX OFFENSE,PUBLIC INDECENCY,STREET,False,False,2534,25.00,36.00,41.91,-87.74
123548,13124301,06/30/2023 12:57:00 AM,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,832,8.00,18.00,41.76,-87.68


In [43]:
years = df['Date'].map(lambda x: x.split()[0].split('/')[-1])
years.value_counts().sort_index()

2001    485885
2002    486804
2003    475983
2004    469421
2005    453772
2006    448177
2007    437087
2008    427169
2009    392824
2010    370502
2011    351973
2012    336272
2013    307475
2014    275742
2015    264773
2016    269805
2017    269092
2018    268804
2019    261286
2020    212160
2021    208700
2022    238663
2023    123550
Name: Date, dtype: int64

## Summary

- The chicago crime dataset has now been saved to your repository as csv files. 
- You should save your notebook, commit your work and push to GitHub using GitHub desktop.